In [ ]:
from trulens_eval import TruChain, Feedback, Huggingface, Tru, Select, OpenAI as OpenAIFeedback
from llm_study import SceneSummarizer, SpeechAction, KnowledgeSummarizer, KnowledgeDocTypes
from langchain.llms.openai import OpenAI
from langchain.schema import AgentAction, Document
from langchain import PromptTemplate, LLMChain
import textwrap
from langchain.chains.summarize import load_summarize_chain
from trulens_eval.schema import FeedbackDefinition
from typing import Sequence, Dict
from langchain.callbacks.manager import CallbackManagerForChainRun, CallbackManagerForLLMRun
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts import FewShotPromptTemplate
from trulens_eval.feedback import Groundedness
from trulens_eval.util import JSONPath, Step
from typing import Optional, Sequence, Callable, Any, List
from pydantic import Field
from abc import ABC, abstractmethod
from langchain.chains.base import Chain
from pydantic import BaseModel
from trulens_eval.util import SerialModel
from langchain.callbacks.manager import CallbackManagerForChainRun
from trulens_eval.app import App
from trulens_eval.tru_basic_app import TruBasicApp
from langchain.llms import BaseLLM
from langchain.schema import LLMResult, Generation

In [2]:
from typing import List, Any
from langchain import LLMChain, PromptTemplate
from langchain.llms import BaseLLM
from langchain.schema import LLMResult, Generation
from trulens_eval import TruChain, Feedback, Select, Provider

class MockLLM(BaseLLM):
    """Returns each prompt as it is, without making any API calls"""
    @property
    def _llm_type(self) -> str:
        return "mock_llm"
    def _generate(self, prompts: List[str], stop=None, run_manager=None, **kwargs: Any) -> LLMResult:
        return LLMResult(generations=[[Generation(text=prompt)] for prompt in prompts])

class CustomProvider(Provider):
    def my_feedback_func(self, prompt: str, inputs: dict) -> float:
        prompt = prompt.format(**inputs)
        return float(len(prompt))

feedback = Feedback(CustomProvider().my_feedback_func) \
    .on(Select.App.app.prompt.template) \
    .on(Select.RecordCalls._call.args.inputs)

chain = LLMChain(
    llm=MockLLM(),
    prompt=PromptTemplate.from_template("Tell a {adjective} joke about {subject}"),
)

app = TruChain(chain, app_id="test", feedbacks=[feedback])

app(inputs={"adjective": "sad", "subject": "ducks"})

✅ In my_feedback_func, input prompt will be set to *.__app__.app.prompt.template .
✅ In my_feedback_func, input inputs will be set to *.__record__.app._call.args.inputs .
✅ app test -> default.sqlite
✅ feedback def. feedback_definition_hash_00f76504ae7952a6176c8ed0ccb1c214 -> default.sqlite


{'adjective': 'sad', 'subject': 'ducks', 'text': 'Tell a sad joke about ducks'}

Feedback Function Exception Caught: Traceback (most recent call last):
  File "/home/lariel/workspace/llm-study/.venv/lib/python3.9/site-packages/trulens_eval/feedback.py", line 862, in run
    feedback_call = FeedbackCall(
  File "pydantic/main.py", line 341, in pydantic.main.BaseModel.__init__
pydantic.error_wrappers.ValidationError: 1 validation error for FeedbackCall
args -> inputs
  str type expected (type=type_error.str)



✅ record record_hash_963dc63d23710a3ad4cae1cd7cdd367e from test -> default.sqlite
⏰ feedback feedback_result_hash_a80631eca88622a8e91fddcc5fc36dd5 on record_hash_963dc63d23710a3ad4cae1cd7cdd367e -> default.sqlite


In [ ]:
from trulens_eval import Provider

In [ ]:
tru = Tru()
tru.start_dashboard(force=True)

In [ ]:
class RenderPromptAdapter:
    # func: Callable[[str], Any]
    # __name__: str = "adapter"

    def __init__(self, func: Callable[[str], Any]):
        self.func=func
        self.__name__=func.__name__

    def __call__(self, prompt, inputs):
        print(prompt)
        print(inputs)
        print(prompt.format(**inputs))
        retval = self.func(prompt.format(**inputs))
        print(retval)
        return retval

In [ ]:
a = RenderPromptAdapter(func=Huggingface().not_toxic)

In [ ]:
keys = ["key_sum", "key_ev"]

assignments = itertools.product(["summary"], [{"e1": "event1", "e2": "event2"}])

for assignment in assignments:
    print({k: v for k, v in zip(keys, assignment)})

In [ ]:
import itertools

In [ ]:
itertools.product(["summary"], ["event1", "event2"])

In [ ]:
class CustomProvider(Provider):
    def my_feedback_func(self, prompt: str, inputs: dict) -> float:
        return len(prompt.format(**inputs))

In [ ]:
f_not_toxic = Feedback(Huggingface().not_toxic_v2) \
    .on(Select.App.app.prompt.template) \
    .on(Select.RecordCalls._call.args.inputs)

In [ ]:
my_chain = LLMChain(
    llm=MockLLM(), # OpenAI(model="text-davinci-003", temperature=0.7),
    prompt=PromptTemplate.from_template("Tell a {adjective} joke about {subject}"),
)

In [ ]:
my_chain(inputs={"adjective": "sad", "subject": "ducks"})

In [ ]:
tru_chain = TruChain(
    app=my_chain,
    app_id="JokeTeller-v1",
    metadata={"url": "http://localhost:5001/#/models/joke_teller/versions/1"},
    feedbacks=[f_not_toxic]
)

In [ ]:
app = TruBasicApp(text_to_text=lambda t: f"returning {t}")
_, record = app.call_with_record(input="hello")

In [ ]:
record

In [ ]:
app = TruChain(PassChain(keys=["key1", "key2"]))
_, record = app.call_with_record(inputs={"key1": "val1", "key2": "val2"})

In [ ]:
help(TruBasicApp)

In [ ]:
help(grounded.grounded_statements_aggregator)

In [ ]:
tru = Tru()
tru.start_dashboard(force=True)

In [ ]:
hugs = Huggingface()
openai = OpenAIFeedback()
grounded = Groundedness(OpenAIFeedback())

f_lang_match = Feedback(hugs.language_match).on_input_output()

f_not_toxic = Feedback(hugs.not_toxic).on_output()

f_groundedness_docs = Feedback(grounded.groundedness_measure) \
    .on(Select.Record.app.combine_documents_chain._call.args.inputs.input_documents[:].page_content) \
    .on_output() \
    .aggregate(grounded.grounded_statements_aggregator)

# try the whole prompt too
# .on(Select.RecordCalls._call.args.inputs.summary, Select.RecordCalls._call.args.inputs.events) \
    # .aggregate(grounded.grounded_statements_aggregator) \
f_groundedness_scene = Feedback(grounded.groundedness_measure) \
    .on(Select.RecordCalls._call.args.inputs[["summary", "events"]]) \
    .aggregate(grounded.grounded_statements_aggregator) \
    .on_output()

feedbacks = [f_lang_match, f_not_toxic]

In [ ]:
help(hugs.not_toxic)

In [ ]:
def chain_wrapper(app_id: str, params: dict, feedbacks: Sequence[FeedbackDefinition]):
    def wrapper(chain: LLMChain):
        return TruChain(chain, app_id=app_id, metadata=params, feedbacks=feedbacks)
    return wrapper

In [ ]:
params = {
    "model": "text-davinci-003",
    "temperature": 0.0,
    "summarize_chain_type": "stuff",
}

yoda_chain = LLMChain(
    llm=OpenAI(model=params["model"], temperature=params["temperature"]),
    prompt=FewShotPromptTemplate(
        prefix="Rephrase the sentence in an inverted fashion\n\nExamples:",
        example_prompt=PromptTemplate.from_template("Input: {input}\nOutput: {output}"),
        example_selector=LengthBasedExampleSelector(
            example_prompt=PromptTemplate.from_template("Input: {input}\nOutput: {output}"),
            examples=[
                {"input": "They need to be at school at seven tomorrow", "output": "At seven tomorrow at school they need to be"},
                {"input": "I was programming in Python", "output": "In Python programming was I"},
                {"input": "They were going to school", "output": "To School they were going"},
            ],
        ),
        suffix="Input: {input}\nOutput:",
        input_variables=["input"],
    ),
    verbose=True,
)

scene_summarizer = SceneSummarizer(
    llm=OpenAI(model=params["model"], temperature=params["temperature"]),
    chain_wrapper=chain_wrapper("SceneSummarizer-v1", params, [f_groundedness_scene])
)

knowledge_summarizer = KnowledgeSummarizer(
    llm=OpenAI(model=params["model"], temperature=params["temperature"]),
)

generic_summarize = load_summarize_chain(
    llm=OpenAI(model=params["model"], temperature=params["temperature"]),
    chain_type=params["summarize_chain_type"],
)

truchain_knowledge = TruChain(
    app=knowledge_summarizer,
    app_id='KnowledgeSummarizer-v1',
    feedbacks=feedbacks,
    metadata=params,
)

truchain_summarize = TruChain(
    app=generic_summarize,
    app_id='GenericSummarizer-v1',
    feedbacks=[f_groundedness_docs],
    metadata=params,
)

truyoda_chain = TruChain(
    app=yoda_chain,
    app_id="YodaChain-v1",
    feedbacks=feedbacks,
    metadata=params,
)

In [ ]:
scene_summarizer(
    current_summary="Henri at was at the beach working with a library named Mlflow. His hair was green",
    new_events=[
        SpeechAction(actor="Henri", text="This is tricky. Maybe Sebastian could help me with this"),
        SpeechAction(actor="Alice", text="Sebastian is on vacation I think"),
    ],
)

In [ ]:
truyoda_chain("I would try swimming but it's bad for my hair")

In [ ]:
truchain_summarize([
    KnowledgeDocTypes.scene("I was sitting outside observing the ravens and the rain"),
    KnowledgeDocTypes.scene("I like to sit outside taking care of the cats"),
])

In [ ]:
stop

In [ ]:
truchain("They had dinner already")

In [ ]:
truchain_summarize([
    KnowledgeDocTypes.scene("I was sitting outside observing the ravens and the rain"),
    KnowledgeDocTypes.scene("I like to sit outside taking care of the cats"),
])

In [ ]:
truchain(dict(
    current_summary="Henri was working with a library named Mlflow",
    new_events=[SpeechAction(actor="Henri", text="This is tricky. Maybe Sebastian could help me with this")]
))

In [ ]:
truchain.run